# 📊 Customer Churn Risk Modeling

## Project Overview
This notebook contains the complete analysis for predicting customer churn for a telecommunications company.

**Business Goal**: Predict which customers are likely to leave (churn) so the company can take proactive retention actions.

---

## Part 1: Data Collection and Understanding

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report, roc_curve)
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print('✅ All libraries imported successfully!')

In [ ]:
# Load the dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(f'✅ Dataset loaded: {df.shape[0]:,} customers, {df.shape[1]} columns')

In [ ]:
# Quick look at the data
df.head()

In [ ]:
# Target variable distribution
print('📊 Churn Distribution:')
print(df['Churn'].value_counts())
print(f'\nChurn Rate: {(df["Churn"] == "Yes").mean() * 100:.2f}%')

---
## Part 2: Data Cleaning and Preprocessing

In [ ]:
# Drop customerID
df = df.drop('customerID', axis=1)

# Fix TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Drop rows with missing values
df = df.dropna()

print(f'✅ Data cleaned: {len(df):,} customers, {df.shape[1]} features')

---
## Part 3: Exploratory Data Analysis (EDA)

In [ ]:
# Distribution plots
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df['tenure'], bins=30, color='steelblue', edgecolor='white')
axes[0].set_title('Tenure Distribution', fontweight='bold')
axes[0].set_xlabel('Tenure (months)')

axes[1].hist(df['MonthlyCharges'], bins=30, color='coral', edgecolor='white')
axes[1].set_title('Monthly Charges Distribution', fontweight='bold')
axes[1].set_xlabel('Monthly Charges ($)')

axes[2].hist(df['TotalCharges'], bins=30, color='seagreen', edgecolor='white')
axes[2].set_title('Total Charges Distribution', fontweight='bold')
axes[2].set_xlabel('Total Charges ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Churn by Contract Type
churn_rate_contract = df.groupby('Contract')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)
print('📊 Churn Rate by Contract Type:')
print(churn_rate_contract.sort_values(ascending=False))

plt.figure(figsize=(8, 4))
churn_rate_contract.plot(kind='bar', color=['#e74c3c', '#f39c12', '#2ecc71'])
plt.title('Churn Rate by Contract Type', fontweight='bold')
plt.xlabel('Contract Type')
plt.ylabel('Churn Rate (%)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print('\n💡 Month-to-month contracts have the highest churn!')

---
## Part 4: Class Imbalance Handling

**The Problem**: We have ~73% non-churners vs ~27% churners. This is imbalanced.

**Why it's a problem**: A model could just predict "No churn" for everyone and be 73% accurate, but useless.

**Solution**: We'll use **SMOTE** (Synthetic Minority Oversampling Technique) to create synthetic samples of the minority class.

In [ ]:
# First, prepare data for modeling
# Encode categorical variables

df_encoded = df.copy()

# Identify categorical columns
categorical_cols = df_encoded.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Churn')  # Keep target separate

print(f'Categorical columns to encode: {len(categorical_cols)}')

# Use Label Encoding for each categorical column
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

# Encode target variable
df_encoded['Churn'] = (df_encoded['Churn'] == 'Yes').astype(int)

print('\n✅ Encoding complete!')
df_encoded.head()

In [ ]:
# Split features and target
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')
print(f'\nClass distribution before SMOTE:')
print(y.value_counts())

In [ ]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print('Class distribution AFTER SMOTE:')
print(pd.Series(y_resampled).value_counts())

print('\n💡 Now both classes have equal samples!')

In [ ]:
# Visualize the change
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Before SMOTE
y.value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Before SMOTE', fontweight='bold')
axes[0].set_xlabel('Churn (0=No, 1=Yes)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['No (0)', 'Yes (1)'], rotation=0)

# After SMOTE
pd.Series(y_resampled).value_counts().plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'])
axes[1].set_title('After SMOTE', fontweight='bold')
axes[1].set_xlabel('Churn (0=No, 1=Yes)')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(['No (0)', 'Yes (1)'], rotation=0)

plt.tight_layout()
plt.show()

---
## Part 5: Model Training and Evaluation

We will train two models:
1. **Logistic Regression** - Simple, interpretable baseline
2. **Random Forest** - Tree-based ensemble method

In [ ]:
# Train-Test Split (use resampled data)
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

print(f'Training set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')

In [ ]:
# Scale features (important for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('✅ Features scaled!')

### Model 1: Logistic Regression

In [ ]:
# Train Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test_scaled)
y_prob_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

print('✅ Logistic Regression trained!')

In [ ]:
# Evaluate Logistic Regression
print('📊 Logistic Regression Results:')
print('=' * 50)
print(f'Accuracy:  {accuracy_score(y_test, y_pred_lr):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_lr):.4f}')
print(f'Recall:    {recall_score(y_test, y_pred_lr):.4f}')
print(f'F1-Score:  {f1_score(y_test, y_pred_lr):.4f}')
print(f'ROC-AUC:   {roc_auc_score(y_test, y_prob_lr):.4f}')

### Model 2: Random Forest

In [ ]:
# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)  # Random Forest doesn't need scaling

# Predictions
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

print('✅ Random Forest trained!')

In [ ]:
# Evaluate Random Forest
print('📊 Random Forest Results:')
print('=' * 50)
print(f'Accuracy:  {accuracy_score(y_test, y_pred_rf):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_rf):.4f}')
print(f'Recall:    {recall_score(y_test, y_pred_rf):.4f}')
print(f'F1-Score:  {f1_score(y_test, y_pred_rf):.4f}')
print(f'ROC-AUC:   {roc_auc_score(y_test, y_prob_rf):.4f}')

In [ ]:
# Compare models
print('📊 Model Comparison:')
print('=' * 60)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
lr_scores = [
    accuracy_score(y_test, y_pred_lr),
    precision_score(y_test, y_pred_lr),
    recall_score(y_test, y_pred_lr),
    f1_score(y_test, y_pred_lr),
    roc_auc_score(y_test, y_prob_lr)
]
rf_scores = [
    accuracy_score(y_test, y_pred_rf),
    precision_score(y_test, y_pred_rf),
    recall_score(y_test, y_pred_rf),
    f1_score(y_test, y_pred_rf),
    roc_auc_score(y_test, y_prob_rf)
]

comparison_df = pd.DataFrame({
    'Metric': metrics,
    'Logistic Regression': lr_scores,
    'Random Forest': rf_scores
})
comparison_df

### Cross-Validation

In [ ]:
# Cross-validation to get more reliable estimates
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Logistic Regression CV
lr_cv_scores = cross_val_score(lr_model, X_train_scaled, y_train, cv=cv, scoring='roc_auc')
print(f'Logistic Regression CV ROC-AUC: {lr_cv_scores.mean():.4f} (+/- {lr_cv_scores.std():.4f})')

# Random Forest CV
rf_cv_scores = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='roc_auc')
print(f'Random Forest CV ROC-AUC: {rf_cv_scores.mean():.4f} (+/- {rf_cv_scores.std():.4f})')

print('\n💡 Cross-validation helps us understand how stable our model performance is.')

---
## Part 6: Decision-Based Threshold Analysis

**Default threshold = 0.5**: If probability > 0.5, predict churn.

**But wait!** Different errors have different costs:
- **False Negative (FN)**: Customer will churn, we predict they won't → HIGH COST (lost customer)
- **False Positive (FP)**: Customer won't churn, we predict they will → LOW COST (unnecessary discount)

Let's test different thresholds to minimize total business cost.

In [ ]:
# Define business costs
COST_FN = 500  # Cost of losing a customer (lost revenue)
COST_FP = 50   # Cost of giving unnecessary discount

print('💰 Business Cost Assumptions:')
print(f'   • False Negative (missed churner): ${COST_FN}')
print(f'   • False Positive (unnecessary offer): ${COST_FP}')

In [ ]:
# Test different thresholds
thresholds = [0.3, 0.4, 0.5, 0.6]

# Use Random Forest probabilities (or Logistic Regression)
probabilities = y_prob_rf  # Using RF model

results = []

for thresh in thresholds:
    # Make predictions based on threshold
    y_pred_thresh = (probabilities >= thresh).astype(int)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thresh).ravel()
    
    # Calculate total cost
    total_cost = (fn * COST_FN) + (fp * COST_FP)
    
    # Calculate metrics
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    results.append({
        'Threshold': thresh,
        'True Positives': tp,
        'False Positives': fp,
        'True Negatives': tn,
        'False Negatives': fn,
        'Precision': precision,
        'Recall': recall,
        'Total Cost ($)': total_cost
    })

results_df = pd.DataFrame(results)
print('📊 Threshold Analysis Results:')
results_df

In [ ]:
# Find the optimal threshold
optimal_idx = results_df['Total Cost ($)'].idxmin()
optimal_threshold = results_df.loc[optimal_idx, 'Threshold']

print(f'✅ OPTIMAL THRESHOLD: {optimal_threshold}')
print(f'   This threshold minimizes total business cost to ${results_df.loc[optimal_idx, "Total Cost ($)"]:,.0f}')

print('\n💡 WHY NOT 0.5?')
print('   Because missing a churner costs more than giving an unnecessary discount.')
print('   A lower threshold catches more churners (higher recall) at the cost of more false alarms.')

In [ ]:
# Visualize threshold analysis
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Total Cost by Threshold
axes[0].bar(results_df['Threshold'].astype(str), results_df['Total Cost ($)'], 
            color=['#2ecc71' if t == optimal_threshold else '#3498db' for t in results_df['Threshold']])
axes[0].set_title('Total Business Cost by Threshold', fontweight='bold')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Total Cost ($)')

# Recall by Threshold
axes[1].plot(results_df['Threshold'], results_df['Recall'], 'o-', color='#e74c3c', linewidth=2, markersize=10)
axes[1].set_title('Recall (Churner Detection Rate) by Threshold', fontweight='bold')
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Recall')
axes[1].axvline(x=optimal_threshold, color='green', linestyle='--', label=f'Optimal: {optimal_threshold}')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## Part 7: Error Analysis

Let's understand WHERE our model makes mistakes.

In [ ]:
# Use optimal threshold for final predictions
y_pred_final = (y_prob_rf >= optimal_threshold).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_final)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted: No', 'Predicted: Yes'],
            yticklabels=['Actual: No', 'Actual: Yes'])
plt.title(f'Confusion Matrix (Threshold = {optimal_threshold})', fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f'\n📊 Confusion Matrix Breakdown:')
print(f'   • True Negatives (correctly predicted NO churn): {tn}')
print(f'   • True Positives (correctly predicted churn): {tp}')
print(f'   • False Positives (wrongly predicted churn): {fp}')
print(f'   • False Negatives (missed churners): {fn}')

In [ ]:
# Analyze misclassified cases
# Create a dataframe with predictions
test_analysis = X_test.copy()
test_analysis['Actual'] = y_test.values
test_analysis['Predicted'] = y_pred_final
test_analysis['Probability'] = y_prob_rf

# False Negatives (Missed Churners) - Most costly errors
false_negatives = test_analysis[(test_analysis['Actual'] == 1) & (test_analysis['Predicted'] == 0)]

# False Positives (Unnecessary offers)
false_positives = test_analysis[(test_analysis['Actual'] == 0) & (test_analysis['Predicted'] == 1)]

print(f'📊 Error Analysis:')
print(f'   • False Negatives (missed churners): {len(false_negatives)}')
print(f'   • False Positives (false alarms): {len(false_positives)}')

In [ ]:
# Compare characteristics of False Negatives vs True Positives
true_positives = test_analysis[(test_analysis['Actual'] == 1) & (test_analysis['Predicted'] == 1)]

print('📊 Characteristics of Missed Churners (False Negatives):')
print('\nMedian values:')
print(f"   Tenure: {false_negatives['tenure'].median():.1f} months")
print(f"   Monthly Charges: ${false_negatives['MonthlyCharges'].median():.2f}")

print('\n📊 Characteristics of Correctly Predicted Churners (True Positives):')
print('\nMedian values:')
print(f"   Tenure: {true_positives['tenure'].median():.1f} months")
print(f"   Monthly Charges: ${true_positives['MonthlyCharges'].median():.2f}")

print('\n💡 INSIGHT: Missed churners might have different characteristics than typical churners.')

---
## 📝 Final Summary and Business Recommendations

In [ ]:
print('=' * 70)
print('                    PROJECT SUMMARY')
print('=' * 70)

print('\n📊 DATASET:')
print(f'   • {len(df):,} customers analyzed')
print(f'   • Churn rate: ~27%')

print('\n🔍 KEY FINDINGS FROM EDA:')
print('   • New customers (< 12 months) are most likely to churn')
print('   • Month-to-month contracts have 43% churn rate')
print('   • Fiber optic customers have highest churn')

print('\n🤖 MODEL PERFORMANCE:')
print(f'   • Best model: Random Forest')
print(f'   • ROC-AUC: {roc_auc_score(y_test, y_prob_rf):.4f}')
print(f'   • Optimal threshold: {optimal_threshold}')

print('\n💼 BUSINESS RECOMMENDATIONS:')
print('   1. Focus retention efforts on customers in first 12 months')
print('   2. Incentivize customers to sign longer contracts')
print('   3. Investigate fiber optic service quality issues')
print('   4. Use the model to identify high-risk customers proactively')

print('\n' + '=' * 70)

---

## 🎉 Project Complete!

This notebook covers the complete end-to-end Customer Churn Risk Modeling process:

1. ✅ Data Loading and Understanding
2. ✅ Data Cleaning and Preprocessing
3. ✅ Exploratory Data Analysis
4. ✅ Class Imbalance Handling (SMOTE)
5. ✅ Model Training (Logistic Regression + Random Forest)
6. ✅ Decision-Based Threshold Analysis
7. ✅ Error Analysis

---